### Two-Class SVM vs Two-Class Locally-Deep (LD-SVM/MSR)

Support vactor machines (SVM) are used to predict binary outcomes.  The two differ when they look at the linearity of the classifiers.  SVM's are notorious for bring very computationally taxing and when you add non-linearity to the mix it can become almost unuseable.  If a vanilla SVM using linear classification analysis scheme is not preforming well and you can afford to sacrifice a little accuracy to bring down compuatation time, the a LD-SVM may be a good alternative.  If the linear classifiers are already resulting in a satisfactory accuracy then adding a localy deep network will not provide any tangible results or may even result in worse results.  I would propose that a LD-SVM would have better results when looking at predicting stock prices because those move using a more stochastic process.  LD-SVM may also preform better when the underlying data operates using more complex distributions such as normal, poisson, or t.  A relationship that is generlly more linerarly dependant such as heiht and weight, would be best fitted by a standard SVM.

In [9]:
import pandas as pd
from scipy.io.arff import loadarff

In [17]:
data = loadarff('veh-prime.arff')
data = pd.DataFrame(data[0])

In [18]:
data.head()

,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f27,f28,f29,f30,f31,f32,f33,f34,f35,CLASS
0,0.063,0.160,0.509,-0.967,0.058,0.000,0.874,0.271,1.307,-0.011,...,-0.924,-0.077,0.108,-0.003,0.381,-0.314,0.929,0.184,-0.001,b'noncar'
1,-0.037,-0.325,-0.626,-0.029,0.121,-0.409,-0.002,-0.835,-0.595,-0.253,...,0.270,0.533,0.152,-0.978,0.157,0.011,-0.254,0.453,-0.621,b'noncar'
2,-0.000,1.253,0.833,-0.970,1.516,0.014,-0.378,1.197,0.546,-0.402,...,-0.408,1.550,0.010,-0.652,-0.403,-0.151,0.000,0.049,-0.113,b'car'
3,-0.743,-0.082,-0.626,0.723,-0.006,-0.000,-0.080,-0.297,0.166,0.311,...,0.819,-0.077,-0.099,-0.001,-0.291,1.633,0.686,1.528,-0.000,b'noncar'
4,-0.939,-1.054,-0.140,0.036,-0.766,0.000,-0.272,1.077,5.236,-0.366,...,0.676,0.533,-0.003,0.122,-0.179,-1.449,0.024,-1.698,0.083,b'noncar'


In [24]:
data.CLASS.replace(b'noncar', -1, inplace=True)
data.CLASS.replace(b'car', 1, inplace=True)
data.head()

,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f27,f28,f29,f30,f31,f32,f33,f34,f35,CLASS
0,0.063,0.160,0.509,-0.967,0.058,0.000,0.874,0.271,1.307,-0.011,...,-0.924,-0.077,0.108,-0.003,0.381,-0.314,0.929,0.184,-0.001,-1
1,-0.037,-0.325,-0.626,-0.029,0.121,-0.409,-0.002,-0.835,-0.595,-0.253,...,0.270,0.533,0.152,-0.978,0.157,0.011,-0.254,0.453,-0.621,-1
2,-0.000,1.253,0.833,-0.970,1.516,0.014,-0.378,1.197,0.546,-0.402,...,-0.408,1.550,0.010,-0.652,-0.403,-0.151,0.000,0.049,-0.113,1
3,-0.743,-0.082,-0.626,0.723,-0.006,-0.000,-0.080,-0.297,0.166,0.311,...,0.819,-0.077,-0.099,-0.001,-0.291,1.633,0.686,1.528,-0.000,-1
4,-0.939,-1.054,-0.140,0.036,-0.766,0.000,-0.272,1.077,5.236,-0.366,...,0.676,0.533,-0.003,0.122,-0.179,-1.449,0.024,-1.698,0.083,-1


In [27]:
from sklearn import svm
from sklearn.model_selection import train_test_split
import copy

full_data = copy.deepcopy(data)

target = copy.deepcopy(full_data.CLASS)
full_data.drop('CLASS', inplace = True, axis = 1)

feature_train, feature_test, target_train, target_test = train_test_split(
    full_data, 
    target,
    random_state = 42,
    test_size = 0.2,
    shuffle=True)

In [30]:
clf = svm.SVC(kernel='linear') #Try C=0.1, and/or 'rbf' kernel to see the impact. 
svm_model = clf.fit(feature_train, target_train) 
target_test_pred = svm_model.predict(feature_test)
accuracy = round(float((target_test_pred == target_test).sum())/len(target_test)*100,2)
print("Full Data Set Accuracy=%.2f%%"%accuracy)

Full Data Set Accuracy=92.35%


In [34]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(clf, full_data, target, cv = 10)
print("10 Fold Cross Validation Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

10 Fold Cross Validation Accuracy: 0.93 (+/- 0.05)


In [35]:
col_names = pd.Series(copy.deepcopy(full_data.columns))

In [53]:
for i in range(10):
    trimmed_data = full_data[col_names.sample(10)]
    scores = cross_val_score(clf, trimmed_data, target, cv = 10)
    print("10 Fold Subset Cross Validation Accuracy: %0.2f (+/- %0.2f)"%\
          (scores.mean(), scores.std() * 2))

10 Fold Subset Cross Validation Accuracy: 0.71 (+/- 0.04)
10 Fold Subset Cross Validation Accuracy: 0.88 (+/- 0.07)
10 Fold Subset Cross Validation Accuracy: 0.76 (+/- 0.06)
10 Fold Subset Cross Validation Accuracy: 0.76 (+/- 0.05)
10 Fold Subset Cross Validation Accuracy: 0.82 (+/- 0.09)
10 Fold Subset Cross Validation Accuracy: 0.65 (+/- 0.08)
10 Fold Subset Cross Validation Accuracy: 0.76 (+/- 0.06)
10 Fold Subset Cross Validation Accuracy: 0.82 (+/- 0.09)
10 Fold Subset Cross Validation Accuracy: 0.85 (+/- 0.08)
10 Fold Subset Cross Validation Accuracy: 0.76 (+/- 0.06)


In the above, using a random subset of the data proves to have less accuracy, but generally has comparable standard devition.